1. Polar Feature Transformation (Quantitative Integration)

You can treat returns or RSI angles as positions in polar space — for example, expressing a stock’s momentum and volatility as (r, θ) where:

r = magnitude (e.g., volatility, ATR, price movement intensity)

θ = phase (e.g., direction of RSI or price change cycle)

You can then feed ['Radius', 'MomentumAngle'] into your Gaussian Process or Bayesian regression as model inputs — it’s a compact, geometry-based representation of market dynamics.

In [ ]:
def polar_features(df):
    """
    Converts price and RSI data into polar coordinate features for modeling.
    r = volatility magnitude, θ = directional angle (momentum phase)
    """
    df['Return'] = df['Close'].pct_change()
    df['Volatility'] = df['Return'].rolling(window=14).std()
    df['MomentumAngle'] = np.arctan2(df['Return'], df['Volatility'])  # direction
    df['Radius'] = np.sqrt(df['Return']**2 + df['Volatility']**2)     # magnitude
    return df


2. Polar Chart Visualization (Qualitative Insight)

You could visualize price phase and volatility as a spiral or radial plot to detect cyclical behavior intuitively:
This can be integrated into your create_individual_portfolio_charts() workflow — show both linear (price) and polar (phase) views for each stock.
It’ll reveal cyclic volatility bursts or oscillations that linear plots miss.

In [ ]:
def plot_polar_market_cycle(df, ticker):
    df = polar_features(df)
    plt.figure(figsize=(7, 7))
    ax = plt.subplot(111, polar=True)
    ax.plot(df['MomentumAngle'], df['Radius'], color='teal', alpha=0.7)
    ax.set_title(f"Polar Cycle Representation for {ticker}", va='bottom')
    plt.show()


3. Bayesian Polar Strategy (Decision Integration)

You can even use the angle quadrant to modify trading bias:

θ near 0° → rising with low volatility (momentum build)

θ near 90° → volatility spikes (potential reversal)

θ near 180° → downward with low volatility (trend exhaustion)

θ near 270° → chaos region

Then integrate this into your Bayesian RSI model:
You could then combine this with bayesian_rsi_signal() to get a joint probability that weighs both RSI and polar-phase behavior.

In [ ]:
def bayesian_polar_signal(r, theta):
    probs = {'buy': 1/3, 'hold': 1/3, 'sell': 1/3}
    if theta < np.pi/4 or theta > 7*np.pi/4:  # upward
        probs['buy'] *= 0.7
    elif np.pi/4 < theta < 3*np.pi/4:  # volatile
        probs['hold'] *= 0.7
    elif 3*np.pi/4 < theta < 5*np.pi/4:  # downward
        probs['sell'] *= 0.7
    else:
        probs['hold'] *= 0.6
    total = sum(probs.values())
    for k in probs:
        probs[k] /= total
    return max(probs, key=probs.get), probs
